We are going to recreate some figures for the "Small Synthetic Dataset" from [insert link to paper].

In [1]:
import pennylane as qml
import torch
from torch import nn
from torch.utils.data import DataLoader
from pennylane import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pennylane as pl

device = 'cpu' # Cuda support for PennyLane is possible though

In [2]:
import sys
sys.path.insert(0, "../")
import utils.utils as utils
import models.fourier_models as fm
import models.quantum_models as qm

In [3]:
# generate our regression task
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

n_samples = 200
n_features = 3
n_informative = 3
n_targets = 1
noise = 1.0
random_state = 42
X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_targets=n_targets, noise=noise, random_state=random_state)
X, y = torch.from_numpy(X), torch.from_numpy(y)

# Scale data to interval [-pi/2, pi/2]
X_scaled = utils.data_scaler(X, interval=(-torch.pi/2, torch.pi/2))

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

# torch dataloaders
train_data_list = []
for i in range(len(X_train)):
    data_point = (X_train[i], y_train[i])
    train_data_list.append(data_point)

test_data_list = []
for i in range(len(X_test)):
    data_point = (X_test[i], y_test[i])
    test_data_list.append(data_point)
    
train_dataloader = DataLoader(train_data_list, batch_size=200, shuffle=True)
test_dataloader = DataLoader(test_data_list, batch_size=200, shuffle=True)

In [4]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        def closure():
            optimizer.zero_grad()
            output = model(X)
            loss = loss_fn(output.flatten(), y)
            loss.backward()
            return loss
        
        optimizer.step(closure)

        if batch % 100 == 0:
            loss = loss_fn(model(X).flatten(), y)
            loss, current = loss.item(), batch * len(X)

        return loss

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred.flatten(), y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

# The Quantum Model

In [5]:
n_qubits = n_features
model = qm.QuantumRegressionModel(n_qubits, n_layers=2, n_trainable_block_layers=1)
model.to(device)
loss_fn = nn.MSELoss(reduction='mean') 
optimizer = torch.optim.LBFGS(model.parameters(), lr=0.5, line_search_fn="strong_wolfe")

In [6]:
# Training
NN_loss = []
NN_test_loss = []

NN_loss.append(test(train_dataloader, model, loss_fn))
NN_test_loss.append(test(test_dataloader, model, loss_fn)) 
epochs = 50
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    NN_loss.append(test(train_dataloader, model, loss_fn))
    NN_test_loss.append(test(test_dataloader, model, loss_fn))    
    print("Done!")

Test Error: 
 Accuracy: 0.0%, Avg loss: 10594.479030 

Test Error: 
 Accuracy: 0.0%, Avg loss: 8271.622788 



  0%|                                                    | 0/50 [00:00<?, ?it/s]

Epoch 1
-------------------------------


/home/franz/anaconda3/envs/fourier/lib/python3.8/site-packages/torch/autograd/__init__.py:154: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811806235/work/c10/cuda/CUDAFunctions.cpp:112.)
  Variable._execution_engine.run_backward(


Test Error: 
 Accuracy: 0.0%, Avg loss: 986.991554 



...

Test Error: 
 Accuracy: 0.0%, Avg loss: 147.236787 

Done!
Epoch 4
-------------------------------
Test Error: 
 Accuracy: 0.0%, Avg loss: 270.834888 



  8%|███▌                                        | 4/50 [04:09<47:01, 61.33s/it]

Test Error: 
 Accuracy: 0.0%, Avg loss: 113.545396 

Done!
Epoch 5
-------------------------------


  8%|███▌                                        | 4/50 [04:46<54:57, 71.70s/it]


KeyboardInterrupt: 

In [ ]:
x = np.arange(0, len(NN_loss), 1)
plt.plot(x, NN_loss, label="training loss")
plt.plot(x, NN_test_loss, label="test loss")
plt.xlabel("Epochs")
plt.ylabel("MSE loss")
plt.legend()

# The Classical Surrogate

In [ ]:
# generate frequencies
max_freq = 2
dim = X_scaled[0].shape[0]

W = utils.freq_generator(max_freq, dim)

# we can solve the problem directly as it is a linear least squares problem, 
# giving the bestapproximation w.r.t. the training data
ba_coeffs = utils.fourier_best_approx(W, X_train, y_train) # vector c in paper

ba_loss = utils.loss(W, ba_coeffs, X_train, y_train).item()
ba_test_loss = utils.loss(W, ba_coeffs, X_test, y_test).item()

In [ ]:
# more in the fashion of ML, we can also train on the data with e.g. LBFGS
NN_loss = []
NN_test_loss = []
W.to(device)
model = fm.Fourier_model(W)
model.to(device)
loss_fn = nn.MSELoss(reduction='mean') # equiv. to torch.linalg.norm(input-target)**2
optimizer = torch.optim.LBFGS(model.parameters(), lr=0.5, history_size=50, line_search_fn="strong_wolfe")

epochs = 50
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    NN_loss.append(test(train_dataloader, model, loss_fn))
    NN_test_loss.append(test(test_dataloader, model, loss_fn))    
    print("Done!")

In [ ]:
x = np.arange(0, len(NN_loss), 1)
plt.plot(x, NN_loss, color="C0", label="training loss")
plt.plot(x, NN_test_loss, color="C1", label="test loss")
plt.plot(x, np.ones_like(x)*ba_loss, linestyle="dashed", color="C0", label="ba training loss")
plt.plot(x, np.ones_like(x)*ba_test_loss, linestyle="dashed", color="C1", label="ba test loss")
plt.xlabel("Epochs")
plt.ylabel("MSE loss")
plt.legend()